In [1]:
import numpy as np
import pytreenet as ptn

# Util

In [2]:
dict = {'c': 3, 'd': 5, 'c': 4 , 'd': 1}
print(dict)
dict = {'e': 3, 'd': 5, 'c': 4 , 'a': 1}
print(ptn.sort_dictionary(dict))

# copy_object(obj, deep=True)
# compare_lists_by_value(list1, list2)
# fast_exp_action(matrix , vector: np.ndarray,
#                 mode: str = "fastest") : modes = "expm" , "eigsh" , "sparse" 

{'c': 4, 'd': 1}
{'a': 1, 'e': 3, 'c': 4, 'd': 5}


# Tensor_Util

In [3]:
from enum import Enum

class SplitMode(Enum):
    FULL = "full"
    REDUCED = "reduced"
    KEEP = "keep"

    def numpy_qr_mode(self) -> str:
        """
        Returns the string required in the numpy QR decomposition.
        """
        if self is SplitMode.FULL:
            return "complete"
        return "reduced"

    def numpy_svd_mode(self) -> True:
        """
        Returns, if a numpy SVD decomposition computes the full or reduced
         matrices.
        """
        return not self is SplitMode.REDUCED

print(SplitMode.FULL.name)
print(SplitMode.REDUCED.name)
print(SplitMode.KEEP.name)    
print(SplitMode.FULL.numpy_qr_mode())
print(SplitMode.REDUCED.numpy_qr_mode())
print(SplitMode.KEEP.numpy_qr_mode()) 
print(SplitMode.FULL.numpy_svd_mode())
print(SplitMode.REDUCED.numpy_svd_mode())
print(SplitMode.KEEP.numpy_svd_mode())

FULL
REDUCED
KEEP
complete
reduced
reduced
True
False
True


In [4]:
t = ptn.crandn((3,5,4,2))
t1 = t.reshape(12,10)
t2 = t.transpose((0,2,1,3))
t2 = t2.reshape(12,10)
print(np.allclose(t1, t2))
t3 = ptn.tensor_matricization(t,(0,2),(1,3))
print(np.allclose(t3, t2))

False
True


In [5]:
matrix = np.random.randn(60, 20)
q, r = np.linalg.qr(matrix, mode="reduced")
q1, r1 = np.linalg.qr(matrix, mode="complete")
q.shape , r.shape , q1.shape , r1.shape

((60, 20), (20, 20), (60, 60), (60, 20))

In [6]:
matrix = np.random.randn(20, 40)
q, r = np.linalg.qr(matrix, mode="reduced")
q1, r1 = np.linalg.qr(matrix, mode="complete")
q.shape , r.shape , q1.shape , r1.shape

((20, 20), (20, 40), (20, 20), (20, 40))

In [7]:
u , s , v = np.linalg.svd(matrix, full_matrices=True)
u.shape , s.shape , v.shape

((20, 20), (20,), (40, 40))

In [8]:
tensor = ptn.crandn((4,5,3,2,5,2))
q , r = ptn.tensor_qr_decomposition(tensor,(0,2,4),(1,3,5), ptn.SplitMode.FULL)
print(q.shape , r.shape)
q , r = ptn.tensor_qr_decomposition(tensor,(0,2,4),(1,3,5), ptn.SplitMode.REDUCED)
print(q.shape , r.shape)
q , r = ptn.tensor_qr_decomposition(tensor,(0,2,4),(1,3,5), ptn.SplitMode.KEEP)
print(q.shape , r.shape)

(4, 3, 5, 60) (60, 5, 2, 2)
(4, 3, 5, 20) (20, 5, 2, 2)
(4, 3, 5, 20) (20, 5, 2, 2)


In [9]:
tensor = ptn.crandn((4,5,3,2,5,2))
u,s,v = ptn.tensor_svd(tensor,(0,2,5),(1,3,4), SplitMode.FULL)
print(u.shape,s.shape,v.shape)
u,s,v = ptn.tensor_svd(tensor,(0,2,5),(1,3,4), SplitMode.REDUCED)
print(u.shape,s.shape,v.shape)
u,s,v = ptn.truncated_tensor_svd(tensor, (0,2,5), (1,3,4),
                         max_bond_dim=np.inf , rel_tol=-np.inf, total_tol=-np.inf)
print(u.shape,s.shape,v.shape)
u,sv = ptn.contr_truncated_svd_splitting(tensor, (0,2,5), (1,3,4),
                                         max_bond_dim=np.inf , rel_tol=-np.inf, total_tol=-np.inf)
print(u.shape,sv.shape)

(4, 3, 2, 24) (24,) (50, 5, 2, 5)
(4, 3, 2, 24) (24,) (24, 5, 2, 5)


TypeError: truncated_tensor_svd() got an unexpected keyword argument 'max_bond_dim'

In [ ]:
matrix = ptn.crandn((40,70))
u, s, vh = np.linalg.svd(matrix, full_matrices=True)
print(u.shape,s.shape,vh.shape)
matrix_reconstruction = np.dot(u, np.dot(np.diag(s), vh[:len(s),:])) 
np.allclose(matrix, matrix_reconstruction)

(40, 40) (40,) (70, 70)


True

# GraphNode

In [ ]:
#1
node = ptn.GraphNode()
print("1" , node.identifier , node.parent, node.children)
#2
node = ptn.GraphNode(identifier="this")
print("2 " ,node.neighbouring_nodes())
print(" " , node.is_root() , node.is_leaf())
#3
node.add_parent("parent")
node.add_child("child1")
node.add_children(["child2","child3"])
print("3 " , node.neighbouring_nodes())
#4
node.remove_child("child3")
node.remove_parent()
print("4 " , node.neighbouring_nodes())
print(" " , node.is_root() , node.is_leaf())
print(" ", node.child_index("child1") , node.neighbour_index("child1") , node.neighbour_index("child2"))
#5
node.add_parent("parent")
print( node.neighbouring_nodes())
print("5 " ,node.nparents() , node.nchildren() , node.nneighbours())
print(" ", node.child_index("child1") , node.neighbour_index("child1") , node.neighbour_index("child2"))
#6
node.replace_child("child1", "C1")
print( "6 " ,node.neighbouring_nodes())
#7
node.replace_neighbour("parent","P1" )
node.replace_neighbour("child2","C2" )
print("8 " , node.neighbouring_nodes())

1 1e707abb-f915-11ee-b3b9-28dfeb1555e4 None []
2  []
  True True
3  ['parent', 'child1', 'child2', 'child3']
4  ['child1', 'child2']
  True False
  0 0 1
['parent', 'child1', 'child2']
5  1 2 3
  0 1 2
6  ['parent', 'C1', 'child2']
8  ['P1', 'C1', 'C2']


In [ ]:
#8
print( node.is_child_of("P1") , node.is_parent_of("C2"))
print( node.has_x_children(2))

True True
True


# Node

In [ ]:
node = ptn.Node(ptn.crandn((3,2,6)) ,"this")
node.add_parent("P1")
node.add_children(["C1", "C2"])
print(node.leg_permutation , node.shape , node.nlegs()) # the erders are well adjusted. 
print(node.parent, node.children)
print(node.parent_leg)
print(node.children_legs)
print(node.get_neighbour_leg("P1") , node.get_neighbour_leg("C1") , node.get_neighbour_leg("C2"))
print(node.get_child_leg("C1") , node.get_child_leg("C2"))
print(".................")
node.remove_parent()
print(node.parent, node.children)
print(node.parent_leg) # always 0
print(node.children_legs)
print(node.get_neighbour_leg("C1") , node.get_neighbour_leg("C2"))
print(node.get_child_leg("C1") , node.get_child_leg("C2"))
node.add_parent("P1")

[0, 1, 2] (3, 2, 6) 3
P1 ['C1', 'C2']
0
[1, 2]


AttributeError: 'Node' object has no attribute 'get_neighbour_leg'

In [ ]:
# 1 
print (" 1) " , node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs , node.nopen_legs())
# 2
node.remove_parent()
print (" 2) " , node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs , node.nopen_legs())
# 3
node.link_tensor(ptn.crandn((3,2,4,5,6))) # replace tensor
print (" 3) " , node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs , node.nopen_legs())
print(len(node.leg_permutation)-node.nneighbours())
node.nchildren()

 1)  [0, 1, 2] ['P1', 'C1', 'C2']
[] 0
 2)  [0, 1, 2] ['C1', 'C2']
[2] 1
 3)  [0, 1, 2, 3, 4] ['C1', 'C2']
[2, 3, 4] 3
3


2

In [ ]:
node.open_leg_to_parent("P1",3)
print (node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs ,node.nopen_legs())
print([node.leg_permutation[i] for i in node.open_legs])
print(node.parent_leg , node.children_legs)
print(node.get_child_leg("C1") , node.get_child_leg("C2"))

[3, 0, 1, 2, 4] ['P1', 'C1', 'C2']
[3, 4] 2
[2, 4]
0 [1, 2]
1 2


In [ ]:
node.open_leg_to_child("C3",3)
print (node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs , node.nopen_legs())
print([node.leg_permutation[i] for i in node.open_legs])
print(node.parent_leg , node.children_legs)
print(node.get_child_leg("C1") , node.get_child_leg("C2") , node.get_child_leg("C3"))
#  open_legs_to_children(self, child_dict: Dict[str, int]):


[3, 0, 1, 2, 4] ['P1', 'C1', 'C2', 'C3']
[4] 1
[4]
0 [1, 2, 3]
1 2 3


In [ ]:
node.parent_leg_to_open_leg()
print (node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs , node.nopen_legs())
print([node.leg_permutation[i] for i in node.open_legs])
print(node.parent_leg , node.children_legs)
print(node.get_child_leg("C1") , node.get_child_leg("C2") , node.get_child_leg("C3"))

[0, 1, 2, 4, 3] ['C1', 'C2', 'C3']
[3, 4] 2
[4, 3]
0 [0, 1, 2]
0 1 2


In [ ]:
node.child_leg_to_open_leg("C2")
print (node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs , node.nopen_legs())
print([node.leg_permutation[i] for i in node.open_legs])
print(node.parent_leg , node.children_legs)
print(node.get_child_leg("C1") , node.get_child_leg("C3"))
9# children_legs_to_open_legs(self, children_id_list: List[str])

[0, 2, 4, 3, 1] ['C1', 'C3']
[2, 3, 4] 3
[4, 3, 1]
0 [0, 1]
0 1


9

In [ ]:
node = ptn.Node(ptn.crandn((5,4,3,6,2,6,7,9,8)))
node.add_parent("P1")
node.add_children(["C1", "C2"])
print (node.leg_permutation , node.neighbouring_nodes())
print(node.open_legs)
# exchanges any leg 
node.exchange_open_leg_ranges(range(2,3),range(5,7))
print(node.leg_permutation , node.open_legs)
print([node.leg_permutation[i] for i in node.open_legs])

[0, 1, 2, 3, 4, 5, 6, 7, 8] ['P1', 'C1', 'C2']
[3, 4, 5, 6, 7, 8]
[0, 1, 5, 6, 3, 4, 2, 7, 8] [3, 4, 5, 6, 7, 8]
[6, 3, 4, 2, 7, 8]


In [ ]:
print (node.leg_permutation , node.neighbouring_nodes())
node.swap_two_child_legs(node.children[0], node.children[1])
print (node.leg_permutation , node.neighbouring_nodes())
node.add_child("C3")
print (node.leg_permutation , node.neighbouring_nodes())
node.swap_with_first_child(node.children[2])
print (node.leg_permutation , node.neighbouring_nodes())


[0, 1, 5, 6, 3, 4, 2, 7, 8] ['P1', 'C1', 'C2']
[0, 5, 1, 6, 3, 4, 2, 7, 8] ['P1', 'C2', 'C1']
[0, 5, 1, 6, 3, 4, 2, 7, 8] ['P1', 'C2', 'C1', 'C3']
[0, 6, 1, 5, 3, 4, 2, 7, 8] ['P1', 'C3', 'C1', 'C2']


In [ ]:
# total open dimension
node.open_dimension()


2592

In [ ]:
node = ptn.Node(ptn.crandn((5,4,3,6,2,6)), "T")
node.add_parent("P1")
node.add_children(["C1", "C2"])
print(node.parent_leg , node.children_legs , node.open_legs )

0 [1, 2] [3, 4, 5]


# LegSpecification

In [ ]:
Legs1 = ptn.LegSpecification(node.parent, node.children, node.open_legs, node)
print(Legs1.parent_leg , Legs1.child_legs , Legs1.open_legs )
print(node.parent , node.children , node.open_legs )
print(node.parent_leg , node.children_legs , node.open_legs )
print(Legs1.__str__() , )
print(Legs1.find_leg_values() , Legs1.find_all_neighbour_ids())

P1 ['C1', 'C2'] [3, 4, 5]
P1 ['C1', 'C2'] [3, 4, 5]
0 [1, 2] [3, 4, 5]
parent_leg: P1, child_legs: ['C1', 'C2'], open_legs: [3, 4, 5], node_id: T, is_root: False
[0, 1, 2, 3, 4, 5] ['P1', 'C1', 'C2']


# TreeStructure 

In [24]:
identifiers = ["node" + str(i) for i in range(9)]
nodes =  [ptn.Node(identifier=identifiers[i])
                 for i in range(len(identifiers))]
print(nodes[0].identifier)
nodes[0]

node0


In [27]:
ts=ptn.TreeStructure()
print(ts._nodes , ts.root_id )
ts.add_root(nodes[0])
print(ts._nodes , ts.root_id )


{} None
{'node0': <pytreenet.core.node.Node object at 0x000001C0D0367470>} node0


In [28]:
print(ts._nodes)
ts.add_child_to_parent(nodes[1], nodes[0].identifier)
print(nodes[1].parent , nodes[0].children )
print(nodes[0].is_parent_of(nodes[1].identifier) , nodes[1].is_child_of(nodes[0].identifier) )
print(ts.is_parent_of(nodes[0].identifier, nodes[1].identifier) , ts.is_child_of(nodes[1].identifier, nodes[0].identifier))
print(ts._nodes)

{'node0': <pytreenet.core.node.Node object at 0x000001C0D0367470>}
node0 ['node1']
True True
True True
{'node0': <pytreenet.core.node.Node object at 0x000001C0D0367470>, 'node1': <pytreenet.core.node.Node object at 0x000001C0D0364590>}


In [29]:
ts.add_parent_to_root(nodes[2])
print(nodes[2].is_root() , nodes[1].is_leaf() )
print(nodes[2].is_parent_of(nodes[0].identifier))


True True
True


In [ ]:
ts.add_child_to_parent(nodes[3], nodes[0].identifier)
ts.add_child_to_parent(nodes[4], nodes[0].identifier)
ts.add_child_to_parent(nodes[5], nodes[4].identifier)
print(ts.nearest_neighbours())
print(ts.get_leaves())
print(ts.leaves_under_node(nodes[4].identifier))

[('node0', 'node1'), ('node0', 'node3'), ('node0', 'node4'), ('node2', 'node0'), ('node4', 'node5')]
['node1', 'node3', 'node5']
{'node5': <pytreenet.node.Node object at 0x00000164D1C53CB0>}


In [ ]:
ts.distance_to_node("node2")

{'node2': 0, 'node0': 1, 'node1': 2, 'node3': 2, 'node4': 2, 'node5': 3}

In [ ]:
print(ts.find_subtree_of_node("node2") == ts._nodes)
ts.find_subtree_of_node("node0")

True


{'node0': <pytreenet.node.Node at 0x164dfecf410>,
 'node1': <pytreenet.node.Node at 0x164d1343320>,
 'node3': <pytreenet.node.Node at 0x164d1c53bc0>,
 'node4': <pytreenet.node.Node at 0x164d1c53950>,
 'node5': <pytreenet.node.Node at 0x164d1c53cb0>}

In [ ]:
print(ts.find_subtree_size_of_node("node2"))
print(ts.find_subtree_size_of_node("node0"))
print(ts.find_subtree_size_of_node("node1"))

6
5
1


In [ ]:
ts.determine_parentage("node0", "node2") # output = (parent_id, child_id)

('node2', 'node0')

In [ ]:
from copy import deepcopy
ts1 = deepcopy(ts)
ts2 = deepcopy(ts)

In [ ]:
#                         node2
#                           |
#               _________ node0 ________          
#               |           |          |
#               node1     node3      node4
#                                      |
#                                    node5

In [ ]:
ts1.find_path_to_root("node5")

['node5', 'node4', 'node0', 'node2']

In [ ]:
print(ts1.path_from_to("node1", "node3"))
print(ts1.path_from_to("node2", "node1"))

['node1', 'node0', 'node3']
['node2', 'node0', 'node1']


In [ ]:
ts1.replace_node_in_neighbours("new", "node0")
print(ts1.nodes["node2"].children)
print(ts1.nodes["node1"].parent)
print(ts1.nodes["node3"].parent)
print(ts1.nodes["node4"].parent)

# node0 is removed 
# ts1.nodes["new"].parent ---> error
# ts1.nodes["new"].children ---> error 

['new']
new
new
new


In [ ]:
ts2.replace_node_in_some_neighbours("new", "node0", ["node2", "node1", "node3"])
print(ts2.nodes["node2"].children)
print(ts2.nodes["node1"].parent)
print(ts2.nodes["node3"].parent)
print(ts2.nodes["node4"].parent)
print(ts2.nodes["node0"].children)
print(ts2.nodes["node0"].parent)
# print(ts2.nodes["new"].parent) ---> error
# print(ts2.nodes["new"].children) ---> error

['new']
new
new
node0
['node1', 'node3', 'node4']
node2


In [ ]:
print(ts2.nodes["node3"].is_child_of("node0"))
print(ts2.nodes["node0"].is_parent_of("node3"))

False
True


# TTN

In [ ]:
#                         node2
#                           |
#               _________ node0 ________          
#               |           |          |
#               node1     node3      node4
#                                      |
#                                    node5

In [14]:
ttn = ptn.TreeTensorNetwork()

node0, tensor0 = ptn.random_tensor_node((1, 2, 3, 4), identifier="node0")

ttn.add_root(node0, tensor0)
print(ttn.tensors["node0"].shape)
print(ttn.nodes["node0"] == node0)
print("node0 : " , node0.parent , node0.children )

(1, 2, 3, 4)
True
node0 :  None []


In [15]:
node2, tensor2 = ptn.random_tensor_node((1,2,3,4,5), identifier="node2")

ttn.add_parent_to_root( 2, node2 , tensor2 , 2)

print("node0 : " , node0.shape )
print("node2 : " , node2.shape )

node0 :  (3, 1, 2, 4)
node2 :  (3, 1, 2, 4, 5)


In [16]:
node1, tensor1 = ptn.random_tensor_node((4, 5, 6, 3), identifier="node1")
node3, tensor3 = ptn.random_tensor_node((3, 8, 2, 4), identifier="node3")
node4, tensor4 = ptn.random_tensor_node((3, 6, 1, 5), identifier="node4")
node5, tensor5 = ptn.random_tensor_node((4, 6, 2, 3), identifier="node5")

ttn.add_child_to_parent( node1 , tensor1 , 0  , "node0" , 3)
ttn.add_child_to_parent( node3 , tensor3 , 2  , "node0" , 3)
ttn.add_child_to_parent( node4 , tensor4 , 2  , "node0" , 3)
ttn.add_child_to_parent( node5 , tensor5 , 1  , "node4" , 2)

In [ ]:
from copy import deepcopy
ttn2 = deepcopy(ttn)
contracted_ttn , contraction_oder =  ttn2.completely_contract_tree(to_copy=True)
print(contracted_ttn.shape) 
print(contraction_oder)
print(ttn2.tensors["node2"].shape)
print(ttn2.nodes["node2"].open_legs)


(1, 2, 4, 5, 5, 6, 3, 3, 8, 4, 3, 5, 4, 2, 3)
['node2', 'node0', 'node1', 'node3', 'node4', 'node5']
(3, 1, 2, 4, 5)
[1, 2, 3, 4]


In [12]:
ttn=ptn.random_small_ttns()
contracted_ttn , contraction_oder =  ttn.completely_contract_tree(to_copy=False)
print(contracted_ttn.shape) 
print(contraction_oder)
print(ttn.nodes)
# ttn.tensors["root"] = contracted_ttn

(2, 3, 4)
['root', 'c1', 'c2']
{'root': <pytreenet.core.node.Node object at 0x000001C0B1522480>}


In [ ]:
ttn2 = deepcopy(ttn)

spec_q = ptn.LegSpecification("node2", ["node1","node4"], [], None)
spec_r = ptn.LegSpecification(None, ["node3"], [], None)
ttn2.split_node_qr("node0" , spec_q , spec_r,"node0_q", "node0_r")

print(ttn2.nodes["node0_r"].neighbouring_nodes())
print(ttn2.nodes["node4"].neighbouring_nodes())
print(ttn2.nodes["node3"].neighbouring_nodes())

In [18]:
from pytreenet.core.canonical_form import _build_qr_leg_specs
spec_q , spec_r = _build_qr_leg_specs(ttn.nodes["node0"],"node2")
print (spec_q.parent_leg , spec_q.child_legs , spec_q.open_legs) 
print (spec_r.parent_leg , spec_r.child_legs , spec_r.open_legs) 

None ['node1', 'node3', 'node4'] []
node2 [] []


In [ ]:
print(node4.open_legs) 
print(node4.neighbouring_nodes())
print(node1.is_child_of(node0.identifier))
print(node3.nlegs() , node3.nopen_legs())
print(ttn.tensors["node4"].shape == node4.shape)
# node , tensor = ttn["node identifier"]
# ttn.root[0] : Node
# ttn.root[1] : tensor
ttn.root[0].identifier == ttn.root_id


[2, 3]
['node0', 'node5']
True
4 3
True


True

In [ ]:
print(ttn.tensors["node4"].shape , ttn.nodes["node4"].parent , ttn.nodes["node4"].children)
print(ttn.tensors["node5"].shape , ttn.nodes["node5"].parent , ttn.nodes["node5"].children)

(1, 6, 3, 5) node0 ['node5']
(6, 4, 2, 3) node4 []


In [ ]:
spec4, spec5 = ttn.legs_before_combination("node4","node5")

print (spec4.parent_leg , spec4.child_legs , spec4.open_legs) 
print (spec5.parent_leg , spec5.child_legs , spec5.open_legs) 

node0 [] [1, 2]
None [] [3, 4, 5]


In [ ]:
ttn.contract_nodes("node4", "node5", "node45")
print(ttn.tensors["node45"].shape , ttn.nodes["node45"].parent , ttn.nodes["node45"].children)


(1, 3, 5, 4, 2, 3) node0 []


In [ ]:
ttn.split_node_svd("node45",spec4, spec5, "node4", "node5")

In [ ]:
print(ttn.tensors["node4"].shape , ttn.nodes["node4"].parent , ttn.nodes["node4"].children)
print(ttn.tensors["node5"].shape , ttn.nodes["node5"].parent , ttn.nodes["node5"].children)

(1, 6, 3, 5) node0 ['node5']
(6, 4, 2, 3) node4 []


In [ ]:
spec0, spec4 = ttn.legs_before_combination("node0","node4")

print (spec0.parent_leg , spec0.child_legs , spec0.open_legs) 
print (spec4.parent_leg , spec4.child_legs , spec4.open_legs) 

node2 ['node1', 'node3'] []
None ['node5'] [4, 5]


In [ ]:
ttn.contract_nodes("node0", "node4", "node0contrnode4")

In [ ]:
spec0.node = ttn.nodes["node0contrnode4"]
spec4.node = ttn.nodes["node0contrnode4"]

spec0.find_leg_values() , spec4.find_leg_values()

([0, 1, 2], [3, 4, 5])

In [ ]:
print( ttn.nodes["node0contrnode4"].shape , ttn.nodes["node0contrnode4"].parent , ttn.nodes["node0contrnode4"].children)
# `(parent_parent_leg, node1_children_legs,
#   node2_children_legs,node1_open_legs, node2_open_legs)`

(3, 4, 2, 6, 3, 5) node2 ['node1', 'node3', 'node5']


In [ ]:
ttn.split_node_qr("node0contrnode4",spec0, spec4, "node0", "node4")

In [ ]:
print(ttn.tensors["node0"].shape , ttn.nodes["node0"].parent , ttn.nodes["node0"].children)
print(ttn.tensors["node4"].shape , ttn.nodes["node4"].parent , ttn.nodes["node4"].children)

(3, 24, 4, 2) node2 ['node4', 'node1', 'node3']
(24, 6, 3, 5) node0 ['node5']


In [ ]:
# conjugated version of the current TTN
ttn = ttn.conjugate() 

In [ ]:
print(ttn.tensors["node3"].shape  , ttn.nodes["node3"].open_legs)
tensor = ptn.crandn((3,8,4,6,7,8))
ttn.absorb_into_open_legs("node3",tensor)
print(ttn.tensors["node3"].shape  , ttn.nodes["node3"].open_legs)

(2, 3, 8, 4) [1, 2, 3]
(2, 6, 7, 8) [1, 2, 3]


In [ ]:
    """
    Generates a small TreeTensorNetworkState of three nodes:
    The root (`"root"`) and its two children (`"c1"` and `"c2"`). The associated 
    tensors are random, but their dimensions are set.

                |2
                |
                r
               / \\
         3|  5/  6\\   |4
          |  /     \\  |
           c1        c2
    """

'\nGenerates a small TreeTensorNetworkState of three nodes:\nThe root (`"root"`) and its two children (`"c1"` and `"c2"`). The associated \ntensors are random, but their dimensions are set.\n\n            |2\n            |\n            r\n           / \\\n     3|  5/  6\\   |4\n      |  /     \\  |\n       c1        c2\n'

In [ ]:
tensortree = ptn.random_small_ttns()

# The tensor to be absorbed has to have twice 
# as many open legs as the node tensor.
tensor = ptn.crandn((2,10))
tensortree.absorb_into_open_legs("root",tensor)
print(tensortree.tensors["root"].shape)

# absorb a matrix to any legs 
tensortree.absorb_matrix("root" , ptn.crandn((6,6)) ,1,0 )


tensortree_cntr = tensortree.completely_contract_tree(to_copy = True)
tensortree_cntr["root"][1].shape , tensortree["root"][1].shape

(5, 6, 10)


AssertionError: Only square Matrices can be absorbed!
If you desire to contract a non-square matrix/ a higher-degree tensor
then add it as a new child to this TTN and contract it.

In [ ]:
tensortree = ptn.random_small_ttns()

q_legs = ptn.LegSpecification(None, ["c1"], [2], None)
r_legs = ptn.LegSpecification(None, ["c2"], [], None)
tensortree.split_node_qr("root", q_legs, r_legs,
                               q_identifier="q1", r_identifier="r1")

In [ ]:
tensortree = ptn.random_small_ttns()
tensortree.canonical_form("c1")
contr_indices = tuple([tensortree.nodes["root"].child_index("c2")] + tensortree.nodes["root"].open_legs)
identity = ptn.compute_transfer_tensor(tensortree.tensors["root"],contr_indices)
np.allclose(identity,np.eye(5))

True

In [ ]:
#                          id1 ___________________
#                           |                     |
#                _________ id2 ________          id8  
#               |                      |          |        
#       _____  id5                    id3        id9 
#      |        |                      |          
#      id7     id6                    id4           

In [ ]:
ttn = ptn.TreeTensorNetwork()
node1, tensor1 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id1")
node2, tensor2 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id2")
node3, tensor3 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id3")
node4, tensor4 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id4")
node5, tensor5 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id5")
node6, tensor6 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id6")
node7, tensor7 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id7")
node8, tensor8 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id8")
node9, tensor9 = ptn.random_tensor_node((2, 3, 4, 5), identifier="id9")

ttn.add_root(node1, tensor1)
ttn.add_child_to_parent(node2, tensor2, 0, "id1", 0)
ttn.add_child_to_parent(node3, tensor3, 3, "id2", 3)
ttn.add_child_to_parent(node4, tensor4, 0, "id3", 1)
ttn.add_child_to_parent(node5, tensor5, 2, "id2", 3)
ttn.add_child_to_parent(node6, tensor6, 0, "id5", 1)
ttn.add_child_to_parent(node7, tensor7, 1, "id5", 2)
ttn.add_child_to_parent(node8, tensor8, 1, "id1", 1)
ttn.add_child_to_parent(node9, tensor9, 2, "id8", 2)

In [ ]:
# canonical form algorithm : 
# distance dictionary : {node_id : distance to the orthogonality center}
# for : "node" with maximum distance , .... , distance = 1 
#     for : "next_node" = neighbour of node with minimum distance in distance dictionary
#           split_qr_contract_r_to_neighbour(node, next_node)

In [ ]:
ttn.tensors["id2"].shape[ttn.nodes["id2"].neighbour_index("id5")]

4

In [ ]:
ttn.canonical_form("id5") # or ttn.orthogonalize("id5")

# check "id2" isometry 
contr_indices = tuple(  [ttn.nodes["id2"].neighbour_index("id1")]
                      + [ttn.nodes["id2"].neighbour_index("id3")]
                      + ttn.nodes["id2"].open_legs)
identity = ptn.compute_transfer_tensor(ttn.tensors["id2"], contr_indices )
np.allclose(identity, np.eye(ttn.tensors["id2"].shape[ttn.nodes["id2"].neighbour_index("id5")]))

# False when orthogonality_center_id = id3 or id1 

True

In [ ]:
# check "id1" isometry 
contr_indices = tuple([ttn.nodes["id1"].neighbour_index("id8")]
                      +[ttn.nodes["id1"].neighbour_index("id2")]
                      + ttn.nodes["id1"].open_legs)
identity = ptn.compute_transfer_tensor(ttn.tensors["id1"], contr_indices )
np.allclose(identity, np.eye(ttn.tensors["id1"].shape[ttn.nodes["id1"].neighbour_index("id2")]))

# True only in the orthogonality_center_id direction  

False

# ForkTreeTensorNetwork

In [ ]:
fttn = ptn.ForkTreeTensorNetwork("M","S")

fttn.add_main_chain_node(ptn.crandn((4,3,2,5)))
fttn.add_main_chain_node(ptn.crandn((2,6,5)) , parent_leg=2 ) # contract 0'th leg with parent_leg
fttn.add_main_chain_node(ptn.crandn((6,4,3)) , parent_leg=1 )

fttn.add_sub_chain_node(ptn.crandn((3,5,4)) , subchain_index = 0 , parent_leg=2 )
fttn.add_sub_chain_node(ptn.crandn((5,6,2)) , subchain_index = 0 , parent_leg=1 )
fttn.add_sub_chain_node(ptn.crandn((5,3,6)) , subchain_index = 1 , parent_leg=2 )
"""
M0 --- S0_0 --- S0_1
|
M1 --- S1_0
|
M2
"""
fttn.main_chain , fttn.sub_chains


([<pytreenet.node.Node at 0x1dff2e40cb0>,
 [[<pytreenet.node.Node at 0x1dff53ea780>,
  []])

In [ ]:
print(fttn.main_chain[0].children)
print(fttn.main_chain[1].neighbouring_nodes())

['M1', 'S0_0']
['M0', 'M2', 'S1_0']


# MatrixProductTree

In [ ]:
shapes = [(5, 6), (5, 7, 2, 3), (7, 3), (3, 6, 2), (6, 2, 5), (2, 5)]
tensors = [ptn.crandn(shape) for shape in shapes]
# only in first two tensors 0'th legs are contracted 

In [ ]:
mpt = ptn.MatrixProductTree.from_tensor_list( tensors, root_site=3)
mpt.left_nodes , mpt.root_id , mpt.right_nodes

({'site2': <pytreenet.node.Node at 0x1dff58e9a90>,
  'site1': <pytreenet.node.Node at 0x1dff58e9010>,
  'site0': <pytreenet.node.Node at 0x1dff58ea570>},
 'site3',
 {'site4': <pytreenet.node.Node at 0x1dff58e82c0>,
  'site5': <pytreenet.node.Node at 0x1dff58eaab0>})

In [ ]:
constant_mps = ptn.MatrixProductState.constant_product_state(
               state_value = 0,
               dimension =  4,
               num_sites = 5,
               node_prefix = "site", # default 
               root_site = 0,
               bond_dimensions = [3,5,2,3]) # default = [1,1,1,1]

In [ ]:
print(constant_mps.nodes["site1"].shape)
print(constant_mps.tensors["site0"])

(3, 5, 4)
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
